<a href="https://colab.research.google.com/github/colincockburn/CISC_473_project/blob/main/image_restoration_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Run this cell at start of each session - Change BRANCH name to your working branch.
# if pushing changes to this cell make sure to add them to both notebooks version of it.

import yaml, os
from google.colab import drive

# ==== Team Bootstrap (Colab) ====
# 1) Mount Drive (data only)
drive.mount('/content/drive')

# 2) define consts
REPO_URL  = "https://github.com/colincockburn/CISC_473_project.git"
REPO_DIR  = "/content/drive/MyDrive/projects/CISC_473_project/"
DATA_ROOT = "/content/drive/MyDrive/projects/CISC_473_project_data"

# 3) Clone repo if missing
import os, subprocess, sys
if not os.path.isdir(REPO_DIR):
    os.makedirs(os.path.dirname(REPO_DIR), exist_ok=True)
    print("Cloning repo to", REPO_DIR)
    !git clone "$REPO_URL" "$REPO_DIR"

# 4) Git identity (first time only; uncomment and change to your info)'
# dont push changes

%cd $REPO_DIR
!git config user.email "colincockburn64@gmail.com"
!git config user.name "colincockburn"

# 5) Sync with origin/main and create/update your feature branch
BRANCH = "cc-data-loader"  # e.g., feature/cc-dataloader
if BRANCH:
  !git fetch origin
  !git checkout main
  !git pull origin main
  !git checkout -B "$BRANCH"
else:
  raise Exception("Please update BRANCH var to your desired working branch")

# 6) Sanity checks
import os
assert os.path.isdir(DATA_ROOT), f"Missing data at: {DATA_ROOT}"
print("OK: repo at", REPO_DIR)
print("OK: data at", DATA_ROOT)
!git status


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/projects/CISC_473_project
M	configs/default.yaml
M	notebooks/data_check.ipynb
M	notebooks/training.ipynb
M	src/data.py
Switched to branch 'main'
Your branch is up to date with 'origin/main'.
From https://github.com/colincockburn/CISC_473_project
 * branch            main       -> FETCH_HEAD
Already up to date.
M	configs/default.yaml
M	notebooks/data_check.ipynb
M	notebooks/training.ipynb
M	src/data.py
Switched to and reset branch 'cc-data-loader'
OK: repo at /content/drive/MyDrive/projects/CISC_473_project/
OK: data at /content/drive/MyDrive/projects/CISC_473_project_data
On branch cc-data-loader
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   configs/default.yaml
	modified:   notebooks/data_check.ipynb
	modif

In [9]:
# get configs
with open(f"{REPO_DIR}/configs/default.yaml", "r") as f:
    cfg = yaml.safe_load(f)

In [ ]:
# run training

In [ ]:
# monitor metrics

In [ ]:
# eval model

In [ ]:
# disply results